In [ ]:
!pip install huggingface_hub transformers
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
!pip install datasets
!pip install transformers
import pandas as pd
import numpy as np
from datasets import load_dataset
import json
import torch
import torch.nn as nn
from typing import List, Tuple, Dict
import spacy
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import ast
import re

In [ ]:
csw_translated = load_dataset("RuishiCh0314/ASCEND-mixed-to-chinese-translation")
csw_aligned_df = pd.read_csv('/content/csw_aligned.csv')
with open("data3_chi_complex_finalized.json", 'rb') as f:
    raw_data = f.read()
    data = json.loads(raw_data.decode('utf-8-sig'))  # or try other encodings

data3 = pd.DataFrame(data)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel
from transformers.modeling_outputs import BaseModelOutput
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
import json
import io

a_sample = data3.iloc[:30].copy()
a_sample.head(2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,instruction,input,output
0,保持健康的三个提示。,,以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...
1,三原色是什么？,,三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...


In [ ]:
import torch
import torch.nn as nn
from transformers import MarianMTModel, AutoTokenizer
from tqdm import tqdm

class CodeMixedGenerator(nn.Module):
    def __init__(self, pretrained_model_name, device):
        super().__init__()
        self.device = device
        self.model = MarianMTModel.from_pretrained(pretrained_model_name).to(device)
        self.copy_gate = nn.Linear(self.model.config.d_model, 1).to(device)
        self.vocab_size = self.model.config.vocab_size

    def forward(self, input_ids, attention_mask, labels=None):
        # Ensure attention_mask is 2D
        if attention_mask.dim() == 1:
            attention_mask = attention_mask.unsqueeze(0)
        elif attention_mask.dim() == 3:
            attention_mask = attention_mask.squeeze(1)

        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            output_hidden_states=True,
            return_dict=True
        )

        decoder_hidden_states = outputs.decoder_hidden_states[-1]
        copy_scores = torch.sigmoid(self.copy_gate(decoder_hidden_states))

        logits = outputs.logits
        batch_size, seq_len, _ = logits.shape

        copy_logits = torch.zeros_like(logits)
        input_ids_expanded = input_ids.unsqueeze(1).expand(-1, seq_len, -1)
        copy_logits.scatter_(2, input_ids_expanded, 1.0)

        combined_logits = (1 - copy_scores) * logits + copy_scores * copy_logits

        outputs.logits = combined_logits
        return outputs

    def generate(self, input_ids, attention_mask):
        self.eval()
        with torch.no_grad():
            input_ids = input_ids.to(self.device)
            attention_mask = attention_mask.to(self.device)

            if input_ids.dim() == 1:
                input_ids = input_ids.unsqueeze(0)
            if attention_mask.dim() == 1:
                attention_mask = attention_mask.unsqueeze(0)
            elif attention_mask.dim() == 3:
                attention_mask = attention_mask.squeeze(1)

            outputs = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=5,
                no_repeat_ngram_size=2,
                length_penalty=1.0,
                early_stopping=True,
                output_hidden_states=True,
                return_dict_in_generate=True
            )

            return outputs.sequences

def train_model(model, train_loader, val_loader, num_epochs, device, tokenizer):
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = CodeMixingLoss(tokenizer)

    best_val_loss = float('inf')
    patience, patience_counter = 3, 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for batch in train_pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, labels)
            loss = criterion(outputs.logits, labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            train_pbar.set_postfix({'loss': loss.item()})

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        with torch.no_grad():
            for batch in val_pbar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask, labels)
                loss = criterion(outputs.logits, labels)
                val_loss += loss.item()
                val_pbar.set_postfix({'loss': loss.item()})

        avg_val_loss = val_loss / len(val_loader)
        scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pt')
            print("New best model saved!")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

    return model

class CodeMixingLoss(nn.Module):
    def __init__(self, tokenizer, alpha=0.5):
        super().__init__()
        self.tokenizer = tokenizer
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    def forward(self, logits, labels):
        ce_loss = self.ce_loss(logits.view(-1, logits.size(-1)), labels.view(-1))

        # Calculate the ratio of Chinese to English tokens
        chinese_mask = (labels >= self.tokenizer.vocab_size // 2).float()
        mixing_ratio = chinese_mask.mean(dim=1)

        # Penalize outputs that are too monolingual
        mixing_loss = ((mixing_ratio - 0.5).abs() * 2).mean()

        return ce_loss + self.alpha * mixing_loss


    def generate(self, input_ids, attention_mask):
        self.eval()
        with torch.no_grad():
            input_ids = input_ids.to(self.device)
            attention_mask = attention_mask.to(self.device)

            if input_ids.dim() == 1:
                input_ids = input_ids.unsqueeze(0)
            if attention_mask.dim() == 1:
                attention_mask = attention_mask.unsqueeze(0)

            outputs = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=5,
                no_repeat_ngram_size=2,
                length_penalty=1.0,
                early_stopping=True,
                output_hidden_states=True,
                return_dict_in_generate=True
            )

            return outputs.sequences
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, max_length=128):
        self.data = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        src_encoding = self.tokenizer(
            item['transcription'],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        tgt_encoding = self.tokenizer(
            item['translation'],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': src_encoding['input_ids'].squeeze(),
            'attention_mask': src_encoding['attention_mask'].squeeze(),
            'labels': tgt_encoding['input_ids'].squeeze()
        }

class CombinedDataset(torch.utils.data.Dataset):
    def __init__(self, code_mixed_data, translation_data, tokenizer, max_length=128):
        self.code_mixed_data = code_mixed_data
        self.translation_data = translation_data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.total_length = len(code_mixed_data) + len(translation_data)

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        if idx < len(self.code_mixed_data):
            item = self.code_mixed_data.iloc[idx]
            src_text = item['instruction_zh']
            tgt_text = item['code_mixed']
        else:
            idx = idx - len(self.code_mixed_data)
            item = self.translation_data[idx]
            src_text = item['transcription']
            tgt_text = item['chinese_translation']

        src_encoding = self.tokenizer(
            src_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        tgt_encoding = self.tokenizer(
            tgt_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': src_encoding['input_ids'].squeeze(),
            'attention_mask': src_encoding['attention_mask'].squeeze(),
            'labels': tgt_encoding['input_ids'].squeeze()
        }

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader, random_split


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
model = CodeMixedGenerator("yl31/opus-mt-zh-en-finetuned-cs-instruction", device)

# Load datasets
code_mixed_data = csw_aligned_df  # This should be your DataFrame
translation_dataset = load_dataset("RuishiCh0314/ASCEND-mixed-to-chinese-translation")
translation_data = translation_dataset['train']

print("Code mixed data size:", len(code_mixed_data))
print("Translation data size:", len(translation_data))

# Create combined dataset
combined_data = CombinedDataset(code_mixed_data, translation_data, tokenizer)

# Split combined dataset
train_size = int(0.8 * len(combined_data))
val_size = len(combined_data) - train_size
train_data, val_data = random_split(combined_data, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CodeMixedGenerator("yl31/opus-mt-zh-en-finetuned-cs-instruction", device)
num_epochs = 10
trained_model = train_model(model, train_loader, val_loader, num_epochs, device, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Code mixed data size: 744
Translation data size: 2739


In [ ]:
from transformers import MarianTokenizer, MarianModel
from huggingface_hub import HfApi,hf_hub_download
def save_model_to_hub(trained_model, tokenizer, hf_username, model_name):
    """
    Save a custom model and tokenizer to Hugging Face Hub
    """
    repo_name = f"{hf_username}/{model_name}"
    local_path = "/content/EduCSW_model"
    os.makedirs(local_path, exist_ok=True)

    # Save the model state dict
    model_save_path = os.path.join(local_path, "pytorch_model.bin")
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'base_tokenizer': "Helsinki-NLP/opus-mt-zh-en",  # Save the base tokenizer info
        'finetuned_weights': "yl31/opus-mt-zh-en-finetuned-cs-instruction"  # Save the finetuned weights info
    }, model_save_path)

    # Save a config file
    config = {
        'model_type': 'CodeMixedGenerator',
        'base_tokenizer': "Helsinki-NLP/opus-mt-zh-en",
        'finetuned_weights': "yl31/opus-mt-zh-en-finetuned-cs-instruction"
    }

    import json
    with open(os.path.join(local_path, 'config.json'), 'w') as f:
        json.dump(config, f)

    # Upload to Hugging Face
    api = HfApi()
    try:
        api.create_repo(repo_name, private=True)
    except:
        pass

    api.upload_folder(
        folder_path=local_path,
        repo_id=repo_name,
        commit_message="Upload model weights and configuration"
    )

    # Clean up
    import shutil
    shutil.rmtree(local_path)

    print(f"Model successfully uploaded to https://huggingface.co/{repo_name}")
def load_model_from_hub(hf_username, model_name, device=None):
    """
    Load the custom model and tokenizer from Hugging Face Hub
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    repo_name = f"{hf_username}/{model_name}"

    # Create a temporary directory for downloading
    import tempfile
    with tempfile.TemporaryDirectory() as tmp_dir:
        # Download the files
        api = HfApi()
        api.snapshot_download(repo_id=repo_name, local_dir=tmp_dir)

        # Load the config to get model information
        with open(os.path.join(tmp_dir, 'config.json'), 'r') as f:
            config = json.load(f)

        # Load the base tokenizer
        tokenizer = AutoTokenizer.from_pretrained(config['base_tokenizer'])

        # Load the model
        checkpoint = torch.load(os.path.join(tmp_dir, "pytorch_model.bin"), map_location=device)

        # Initialize model with the same architecture
        model = CodeMixedGenerator(config['finetuned_weights'], device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()

    return model, tokenizer

In [ ]:
save_model_to_hub(
    trained_model=trained_model,
    tokenizer=tokenizer,
    hf_username="RuishiCh0314",
    model_name="EduCSW_model"
)


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

Model successfully uploaded to https://huggingface.co/RuishiCh0314/EduCSW_model


In [ ]:
def generate_code_mixed(input_text, model, tokenizer):
    device = next(model.parameters()).device
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['input_ids'].to(device)
    attention_mask = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['attention_mask'].to(device)

    with torch.no_grad():
        generated_ids = model.generate(input_ids, attention_mask)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Load and use the model
loaded_model, loaded_tokenizer = load_model_from_hub("RuishiCh0314", "EduCSW_model")

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

<ipython-input-50-a601e9abb6c5>:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(tmp_dir, "pytorch_model.bin"), map_location=device)


In [ ]:
loaded_model.eval()

def generate_code_mixed(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['input_ids'].to(device)
    attention_mask = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['attention_mask'].to(device)
    with torch.no_grad():
        generated_ids = loaded_model.generate(input_ids, attention_mask)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)
# Apply the generation function to each row in the DataFrame

In [ ]:
csw_aligned_df_sample = csw_aligned_df.iloc[:30].copy()
csw_aligned_df_sample['code_mixed'] = csw_aligned_df_sample['instruction_zh'].apply(generate_code_mixed)

In [ ]:
csw_aligned_df_sample[['instruction_zh','code_mixed']]

,instruction_zh,code_mixed
0,什么是计算机？,什么 computer?
1,计算机的基本组成部分有哪些？,什么 是 computer? what's the principles?
2,计算机硬件与软件的区别是什么？,计算机 techniques with 软件的 difference what ?
3,什么是冯·诺依曼结构？,什么 von nueman structure?
4,什么是嵌入式系统？,什么 是 engineer system?
5,什么是计算机程序？,什么 是 process?
6,什么是操作系统？,什么 是 operating system?
7,什么是硬盘？,什么 是 dyke?
8,什么是内存？,什么 是 ical?
9,什么是外存？,什么 是 存?


In [ ]:
csw_aligned_df['code_mixed_generated'] = csw_aligned_df['instruction_zh'].apply(generate_code_mixed)

In [ ]:
sample_indices_short = random.sample(range(len(csw_aligned_df)-200), 25)
sample_indices_long = csw_aligned_df.iloc[675:700]
short = csw_aligned_df.iloc[sample_indices_short]
combined_df = pd.concat([short, sample_indices_long], axis=0).reset_index(drop=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   instruction_zh        50 non-null     object 
 1   instruction_en        50 non-null     object 
 2   input_zh              0 non-null      float64
 3   input_en              50 non-null     object 
 4   alignment_info        50 non-null     object 
 5   alignment_dict        50 non-null     object 
 6   code_mixed            50 non-null     object 
 7   code_mixed_generated  50 non-null     object 
dtypes: float64(1), object(7)
memory usage: 3.2+ KB


In [ ]:
csw_aligned_df.to_csv('csw_generated.csv',index=False)
combined_df.to_csv('purposefully_sampled_csw_generated.csv',index=False)

# Evaluation: Qualitative & CMI

In [ ]:
import random
sample_indices = random.sample(range(len(csw_aligned_df)), 5)
for i in sample_indices:
    row = csw_aligned_df.iloc[i]
    print(f"\nRow {i}:")
    print(f"Chinese: {row['instruction_zh']}")
    print(f"Code-mixed: {row['code_mixed']}")
    print(f"Code-mixed_generated: {row['code_mixed_generated']}")
    print("-" * 50)


Row 424:
Chinese: 什么是数据可用性？
Code-mixed: 什么 是 data availability?
Code-mixed_generated: 什么 是 data application?
--------------------------------------------------

Row 697:
Chinese: 在自然语言处理（NLP）中，如何通过文本摘要技术提取文档的关键信息？请描述其工作原理和关键技术。
Code-mixed: 在 language processing （ nlp ）中，如何 through 文本摘要 technology? 提取 documents 的 key information technology? please describe 其工作原理 and technologies.
Code-mixed_generated: 在 language processing ( nlp )中,如何通过文本 analysis technology? 取 the dagrication of the chains? please describe 其工作 principles and tey tyncs.
--------------------------------------------------

Row 267:
Chinese: 什么是区块链的分布式账本？
Code-mixed: 什么 block chain? for distributed account chain?
Code-mixed_generated: 什么 是 block chain? in the different-packed challeng?
--------------------------------------------------

Row 57:
Chinese: 什么是云计算？
Code-mixed: 什么 云 computing?
Code-mixed_generated: 什么 是 cloud calculation?
--------------------------------------------------

Row 445:
Chinese: 什么是存储卷？
Code-mixed:

In [ ]:
import pandas as pd
import re
from typing import List, Tuple

class CMICalculator:
    def __init__(self):
        # Define regex patterns for different types of tokens
        self.chinese_pattern = re.compile(r'[\u4e00-\u9fff]+')
        self.english_pattern = re.compile(r'[a-zA-Z]+')

    def tokenize(self, text: str) -> List[str]:
        """Split text into tokens while preserving Chinese characters"""
        # First split by whitespace for English words
        tokens = text.split()
        processed_tokens = []

        for token in tokens:
            # If token contains Chinese characters, split them
            if self.chinese_pattern.search(token):
                # Split into individual Chinese characters
                chinese_chars = list(token)
                processed_tokens.extend(chinese_chars)
            else:
                processed_tokens.append(token)

        return processed_tokens

    def identify_language(self, token: str) -> str:
        """Identify the language of a token"""
        if self.chinese_pattern.match(token):
            return 'zh'
        elif self.english_pattern.match(token):
            return 'en'
        else:
            return 'other'

    def calculate_cmi(self, text: str) -> Tuple[float, dict]:
        """Calculate Code-Mixing Index and language statistics"""
        tokens = self.tokenize(text)

        # Count tokens by language
        language_counts = {'zh': 0, 'en': 0, 'other': 0}
        for token in tokens:
            lang = self.identify_language(token)
            language_counts[lang] += 1

        # Calculate total meaningful tokens (excluding 'other')
        total_tokens = language_counts['zh'] + language_counts['en']

        # Calculate switches between languages
        switches = 0
        prev_lang = None
        for token in tokens:
            curr_lang = self.identify_language(token)
            if curr_lang != 'other':  # Skip non-linguistic tokens
                if prev_lang and curr_lang != prev_lang:
                    switches += 1
                prev_lang = curr_lang

        # Calculate CMI
        if total_tokens > 0:
            cmi = (switches / total_tokens) * 100
        else:
            cmi = 0.0

        # Prepare detailed statistics
        stats = {
            'total_tokens': total_tokens,
            'language_counts': language_counts,
            'switches': switches,
            'cmi': cmi
        }

        return cmi, stats

def calculate_dataset_cmi(df: pd.DataFrame, text_column: str) -> pd.DataFrame:
    """Calculate CMI for an entire dataset"""
    calculator = CMICalculator()

    results = []
    for idx, row in df.iterrows():
        text = row[text_column]
        cmi, stats = calculator.calculate_cmi(text)

        result = {
            'row_id': idx,
            'cmi': cmi,
            'total_tokens': stats['total_tokens'],
            'chinese_tokens': stats['language_counts']['zh'],
            'english_tokens': stats['language_counts']['en'],
            'other_tokens': stats['language_counts']['other'],
            'language_switches': stats['switches']
        }
        results.append(result)

    return pd.DataFrame(results)

In [ ]:
results_df = calculate_dataset_cmi(csw_aligned_df, 'code_mixed_generated')
np.mean(results_df['cmi'])

25.576600214861916

In [ ]:
results_df_first_round = calculate_dataset_cmi(csw_aligned_df, 'code_mixed')
np.mean(results_df_first_round['cmi'])

26.890539204475054